# 使用Sanic构建异步的RESTful接口服务

[sanic](https://github.com/huge-success/sanic)是一个基于python3.6+新特性协程的异步框架,其接口设计参照了同样受欢迎的[flask](http://flask.pocoo.org/)但它比flask更加轻量,由于协程的作用,其并发性能更强.而且并不需要借助其他http组件就可以单独运行,甚至于它还可以多进程启动.当然相对的它的生态比flask差的多,但作为一个Restful接口服务它已经很够用了,同时他也是目前最成熟的python异步接口的http框架.

## helloworld

我们依然从一个[helloworld]()开始.

RESTful接口基于HTTP的METHOD,因此我们也常用`HTTPMethodView`构造基于METHOD的接口

这种应用构造的过程就是:

+ 使用`Sanic()`创建一个应用实例
+ 继承`HTTPMethodView`构造一个对应RESTful概念中RESOURCE的视图类
+ 通过调用视图类的方法`.as_view()`将视图类转化为一个视图对象
+ 调用应用实例的`.add_route(view,url)`方法将视图对象注册到应用实例对应的url下


+ `app.py`

```python
from sanic import Sanic
from sanic.views import HTTPMethodView
from sanic.response import json as jsonify
from jsonschema import validate

app = Sanic()

User = []

User_Schema = {
    "title": "User",
    "description": "用户",
    "type": "object",
    "properties": {
        "name": {
            "description": "user name",
            "type": "string"
        }, "age": {
            "description": "user age",
            "type": "integer",
            "minimum": 0,
            "maximum": 140,
            "exclusiveMaximum": True
        }
    },
    "required": ["name", "age"]
}


class UserIndexAPI(HTTPMethodView):

    async def get(self, request):
        count = len(User)
        result = {
            "description": "测试api,User总览",
            "user-count": count,
            "links": [
                {
                    "uri": "/user",
                    "method": "POST",
                    "description": "创建一个新用户"
                },
                {
                    "uri": "/user/<int:uid>",
                    "method": "GET",
                    "description": "用户号为<id>的用户信息"
                },
                {
                    "uri": "/user/<int:uid>",
                    "method": "PUT",
                    "description": "更新用户号为<id>用户信息"
                },
                {
                    "uri": "/user/<int:uid>",
                    "method": "DELETE",
                    "description": "删除用户号为<id>用户"
                },
            ]
        }

        return jsonify(result, ensure_ascii=False)

    async def post(self, request):
        insert = request.json

        try:
            validate(instance=insert, schema=User_Schema)
        except Exception as e:
            return jsonify({
                "msg": "参数错误",
                "error": str(e)
            }, ensure_ascii=False), 401
        else:
            uid = User.append(insert)
            return jsonify({
                "msg": "插入成功",
                "uid": uid
            }, ensure_ascii=False)


class UserAPI(HTTPMethodView):

    async def get(self, request, uid):
        try:
            u = User[uid]
        except IndexError as dn:
            return jsonify({
                "msg": "未找到用户",
            }, ensure_ascii=False), 401

        except Exception as e:
            return jsonify({
                "msg": "执行错误",
            }, ensure_ascii=False), 500
        else:
            if u:
                return jsonify(u, ensure_ascii=False)
            else:
                return jsonify({
                    "msg": "未找到用户",
                }, ensure_ascii=False), 401

    async def put(self, request, uid):
        try:
            u = User[uid]
        except IndexError as dn:
            return jsonify({
                "msg": "未找到用户",
            }, ensure_ascii=False), 401

        except Exception as e:
            return jsonify({
                "msg": "执行错误",
            }, ensure_ascii=False), 500
        else:
            if u:
                insert = request.json
                u.update(insert)
                return jsonify({
                    "msg": "更新成功"
                }, ensure_ascii=False)
            else:
                return jsonify({
                    "msg": "未找到用户",
                }, ensure_ascii=False), 401

    async def delete(self, request, uid):
        try:
            u = User[uid]
        except IndexError as dn:
            return jsonify({
                "msg": "未找到用户",
            }, ensure_ascii=False), 401

        except Exception as e:
            return jsonify({
                "msg": "执行错误",
            }, ensure_ascii=False), 500
        else:
            if u:
                User[uid] = None
                return jsonify({
                    "msg": "删除成功",
                }, ensure_ascii=False)
            else:
                return jsonify({
                    "msg": "未找到用户",
                }, ensure_ascii=False), 401


user_index_view = UserIndexAPI.as_view()
user_view = UserAPI.as_view()

app.add_route(user_index_view, '/user')
app.add_route(user_view, '/user/<int:uid>')

if __name__ == "__main__":
    app.run(host='localhost', port=5000)

```

为了防止`json`和标准库重名,我们最好将`sanic.response.json`重命名为`jsonify`.为了返回的数据支持utf-8,我们需要为`jsonify`接口传入参数`ensure_ascii=False`.

这个app可以直接使用命令`python app.py`独立部署.我们可以直接设置参数定义部署时候的服务器信息.主要的参数有:

参数|默认值|说明
---|---|---
`host`|"127.0.0.1"|服务的host
`port`|8000|服务的端口
`debug`| False| debug模式,会降低性能
`ssl` |None|ssl上下文,用于配置https
`sock`|None|可以通过绑定`unix sock`而非host+port来建立连接
`workers`|1|执行服务的进程数
`loop`|None|指定一个asyncio协议的loop对象
`access_log`|True|请求信息记录进log,会显著降低性能





## 为接口写测试



## 使用蓝图为接口分组

## 使用钩子控制服务行为

## 修改log

## 构造插件

## 错误处理